In [150]:
import pandas as pd
import numpy as np
import nltk
import spacy
import difflib
nltk.download('stopwords')
from tqdm import tqdm
from text_to_num import text2num
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer
tqdm.pandas(desc="my bar!")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheusmbr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [151]:
try:
    nlp = spacy.load('pt_core_news_lg')
except:
    import os
    os.system('python -m spacy download pt_core_news_lg')
    nlp = spacy.load('pt_core_news_lg')

In [152]:
year = 2013
df = pd.read_csv(f'./Raw/{year}.csv', sep=';', encoding ='latin')
df_processed = df[['DS_RESUMO', 'DS_PALAVRA_CHAVE', 'NM_AREA_AVALIACAO']].copy()

In [153]:
df_processed.head()

,DS_RESUMO,DS_PALAVRA_CHAVE,NM_AREA_AVALIACAO
0,A POSSIBILIDADE DA OCORRÊNCIA DE SISMOS NO BRA...,"ANÁLISE SÍSMICA, CONCRETO ARMADO, DETALHAMENTO...",ENGENHARIAS I
1,AS OBRAS DE PONTES ESTAIADAS ENVOLVEM UMA TECN...,PONTE ESTAIADA. PONTE DO SABER. ANÁLISE ESTRUT...,ENGENHARIAS I
2,A HIPERPLASIA DE CÉLULAS NEUROENDÓCRINAS DO LA...,1. DOENÇAS PULMONARES INTERSTICIAIS  RADIOGRA...,MEDICINA II
3,ESTUDOS RECENTES TEM DEMONSTRADO ALTERAÇÕES DA...,1. NEURALGIA DO TRIGÊMEO - FISIOPATOLOGIA. 2. ...,MEDICINA II
4,DENTRE AS TÉCNICAS MAIS UTILIZADAS NA DETERMIN...,"PLATAFORMAS FIXAS, ESTRUTURAS OFFSHORE, ESTRUT...",ENGENHARIAS I


In [154]:
df_processed.columns = df_processed.columns.str.slice(3).str.lower()
df_processed.rename(columns={
    'base': 'ano'
}, inplace=True)

for item in list(df_processed.iloc[:, 0:].columns):
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lower()

In [155]:
df_processed.isna().sum()

resumo            0
palavra_chave     0
area_avaliacao    0
dtype: int64

In [156]:
# df_processed.dropna(subset=['abstract'], inplace=True)

# df_processed.fillna({ 'projeto': 'não informado' }, inplace=True)

# df_processed.drop(index=df_processed[
#     df_processed.resumo.str.contains("não informado") | df_processed.palavra_chave.str.contains("não informado") | 
#     df_processed.abstract.str.contains("não informado") | df_processed.keyword.str.contains("não informado") | 
#     df_processed.grande_area_conhecimento.str.contains("não informado")
# ].index, inplace=True)
# df_processed.reset_index(drop=True, inplace=True)

In [157]:
# idx = df_processed[(df_processed.palavra_chave == df_processed.keyword) | (df_processed.resumo == df_processed.abstract) | (df_processed.palavra_chave == df_processed.resumo) | (df_processed.keyword == df_processed.abstract)].index
# df_processed.drop(index=idx, inplace=True)
# df_processed.reset_index(drop=True, inplace=True)
# del idx

In [158]:
def rename_areas(text: str) -> str:
    if text in ['administração pública e de empresas, ciências contábeis e turismo', 'administração, ciências contábeis e turismo']:
        return 'administração pública e de empresas, ciências contábeis e turismo'
    elif text in ['arquitetura e urbanismo', 'arquitetura, urbanismo e design']:
        return 'arquitetura, urbanismo e design'
    elif text in 'filosofia/teologia:subcomissão filosofia':
        return 'filosofia'
    elif text in ['filosofia/teologia:subcomissão teologia', 'teologia']:
        return 'ciências da religião e teologia'
    elif text in 'letras / linguística':
        return 'linguística e literatura'
    elif text in 'artes / música':
        return 'artes'
    elif text in 'ciências sociais aplicadas i':
        return 'serviço social'
    else:
        return text

In [159]:
df_processed.area_avaliacao = df_processed.area_avaliacao.progress_apply(rename_areas)
del rename_areas

my bar!: 100%|██████████| 67534/67534 [00:00<00:00, 1228624.40it/s]


In [160]:
colegios_dict = {
    'ciências da vida': {
        'ciências agrárias': ['ciência de alimentos', 'ciências agrárias i', 'medicina veterinária', 'zootecnia / recursos pesqueiros'],
        'ciências biológicas': ['biodiversidade', 'ciências biológicas i', 'ciências biológicas ii', 'ciências biológicas iii'],
        'ciências da saúde': ['educação física', 'enfermagem', 'farmácia', 'medicina i', 'medicina ii', 'medicina iii', 'nutrição', 'odontologia', 'saúde coletiva']
    },
    'humanidades': {
        'ciências humanas': ['antropologia / arqueologia', 'ciência política e relações internacionais', 'ciências da religião e teologia', 'educação', 'filosofia', 'geografia', 'história', 'psicologia', 'sociologia'],
        'ciências sociais aplicadas': ['administração pública e de empresas, ciências contábeis e turismo', 'arquitetura, urbanismo e design', 'comunicação e informação', 'direito', 'economia', 'planejamento urbano e regional / demografia', 'serviço social'],
        'linguística, letras e artes': ['artes', 'linguística e literatura']
    },
    'ciências exatas, tecnológicas e multidisciplinar': {
        'ciências exatas e da terra': ['astronomia / física', 'ciência da computação', 'geociências', 'matemática / probabilidade e estatística', 'química'],
        'engenharias': ['engenharias i', 'engenharias ii', 'engenharias iii', 'engenharias iv'],
        'multidisciplinar': ['biotecnologia', 'ciências ambientais', 'ensino', 'interdisciplinar', 'materiais']
    }
}

In [161]:
df_processed['colegios'] = ''
df_processed['areas'] = ''
df_processed.rename(columns={'area_avaliacao': 'subareas'}, inplace=True)

for colegios, areas in colegios_dict.items():
    for areas, subareas in areas.items():
        df_processed.loc[df_processed.subareas.isin(subareas), 'colegios'] = colegios
        df_processed.loc[df_processed.subareas.isin(subareas), 'areas'] = areas
        
df_processed = df_processed[['resumo', 'palavra_chave', 'colegios', 'areas', 'subareas']]

In [162]:
df_processed.head()

,resumo,palavra_chave,colegios,areas,subareas
0,a possibilidade da ocorrência de sismos no bra...,"análise sísmica, concreto armado, detalhamento...","ciências exatas, tecnológicas e multidisciplinar",engenharias,engenharias i
1,as obras de pontes estaiadas envolvem uma tecn...,ponte estaiada. ponte do saber. análise estrut...,"ciências exatas, tecnológicas e multidisciplinar",engenharias,engenharias i
2,a hiperplasia de células neuroendócrinas do la...,1. doenças pulmonares intersticiais  radiogra...,ciências da vida,ciências da saúde,medicina ii
3,estudos recentes tem demonstrado alterações da...,1. neuralgia do trigêmeo - fisiopatologia. 2. ...,ciências da vida,ciências da saúde,medicina ii
4,dentre as técnicas mais utilizadas na determin...,"plataformas fixas, estruturas offshore, estrut...","ciências exatas, tecnológicas e multidisciplinar",engenharias,engenharias i


Palavra Chave

In [163]:
df_processed.palavra_chave.replace(to_replace = r'1[.]', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' \d[.]', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'(\S{2,})(?:\.)', value = '\\1;', regex=True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'\b(pt|en)\b', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' ,|, | [,] | /|/ | [/] ', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' [.] |\(\d\)', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' \x96 |\x93', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'[^a-z0-9\s\x80-\xff,.:;?!-]', value = '', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' - |; | ;|; ;', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r';{2,}', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace= r'[\W_]+$|^[;]+', value='', regex=True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'palavras-chave\S', value = '', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'^\s+|\s+$|\x80', value = '', regex = True, inplace=True)

Resumo

In [164]:
df_processed.resumo.replace(to_replace= r'[^a-zA-Z0-9\s]+', value='', regex=True, inplace=True)

In [165]:
key_words = df_processed.palavra_chave.copy()
key_words = list(set(';'.join(key_words).split(';')))
key_words = [item for item in key_words if len(item) > 1]

for i in range (len(key_words)):
    key_words[i] = key_words[i].strip()
    
key_words = [item for item in key_words if len(item) < 80]

In [ ]:
key_words = list(set(key_words))

In [166]:
len(key_words)

118078

In [167]:
def calculate_similarity_or_pertency(sentence: str, text: str) -> str:
    if sentence in text:
        return True
    similaridade = difflib.SequenceMatcher(None, sentence, text).ratio()
    porcentagem = int(similaridade * 100)
    if porcentagem > 80:
        return True
    
    return False

In [168]:
freq_colegio = dict()
freq_area = dict()
freq_subarea = dict()

for character in key_words:
    freq_colegio[character] = {}
    freq_area[character] = {}
    freq_subarea[character] = {}
    
    for item in sorted(df_processed.colegios.unique()):
        freq_colegio[character][item] = 0
        
    for item in sorted(df_processed.areas.unique()):
        freq_area[character][item] = 0
        
    for item in sorted(df_processed.subareas.unique()):
        freq_subarea[character][item] = 0
    
    corte = df_processed[df_processed.palavra_chave.apply(lambda x: calculate_similarity_or_pertency(character, x)) | df_processed.resumo.apply(lambda x: calculate_similarity_or_pertency(character, x))]
    
    colegio = corte.colegios
    area = corte.areas
    subarea = corte.subareas
    
    for item in sorted(colegio.unique()):
        freq_colegio[character][item] = colegio.value_counts()[item]
        
    for item in sorted(area.unique()):
        freq_area[character][item] = area.value_counts()[item]
        
    for item in sorted(subarea.unique()):
        freq_subarea[character][item] = subarea.value_counts()[item]
        
freq_colegio = pd.DataFrame(freq_colegio).transpose()
freq_area = pd.DataFrame(freq_area).transpose()
freq_subarea = pd.DataFrame(freq_subarea).transpose()

In [ ]:
total_colegio = pd.DataFrame(freq_colegio.sum(axis=0), columns=['frequencia'])
total_area = pd.DataFrame(freq_area.sum(axis=0), columns=['frequencia'])
total_subarea = pd.DataFrame(freq_subarea.sum(axis=0), columns=['frequencia'])

In [ ]:
total_area.sort_values(by='frequencia', ascending=False).head(10)

,frequencia
"linguística, letras e artes",363
multidisciplinar,149
ciências agrárias,93
ciências biológicas,82
ciências da saúde,76
ciências exatas e da terra,36
engenharias,32


In [ ]:
freq_colegio['frequencia'] = freq_colegio.sum(axis=1)
freq_area['frequencia'] = freq_area.sum(axis=1)
freq_subarea['frequencia'] = freq_subarea.sum(axis=1)

In [ ]:
freq_colegio.sum(axis=0)
freq_area.sum(axis=0)
freq_subarea.sum(axis=0)

biodiversidade                7
ciência da computação        36
ciências agrárias i          93
ciências biológicas i        75
engenharias i                32
interdisciplinar            149
linguística e literatura    363
medicina i                   20
medicina ii                  56
frequencia                  831
dtype: int64

In [ ]:
freq_colegio.to_pickle('freq_colegio.pkl')
freq_area.to_pickle('freq_area.pkl')
freq_subarea.to_pickle('freq_subarea.pkl')

In [ ]:
# for item in ['resumo', 'abstract', 'palavra_chave', 'keyword']:
#     df_processed[f'{item}'].replace(to_replace ='[0-9]+', value = '', regex = True, inplace=True)
#     df_processed[f'{item}'] = df_processed[f'{item}'].progress_apply(remove_number_words)
#     df_processed[f'{item}'].replace(to_replace ='[,.:;!?]+', value = '', regex = True, inplace=True)
#     df_processed[f'{item}'].replace(to_replace ='[/<>()|\+\-\$%&#@\'\"]+', value = '', regex = True, inplace=True)
#     df_processed[f'{item}'].replace(to_replace ='\W+', value = ' ', regex = True, inplace=True)

In [ ]:
# for item in ['palavra_chave', 'keyword']:
#     df_processed[f'{item}'].replace(to_replace ='([0-9][. ])', value = ',', regex = True, inplace=True)
#     df_processed[f'{item}'].replace(to_replace ='([.])+', value = ',', regex = True, inplace=True)
#     df_processed[f'{item}'].replace(to_replace ='(  )+', value = ' ', regex = True, inplace=True)
#     df_processed[f'{item}'] = df_processed[f'{item}'].str.lstrip(', ')

In [ ]:
# df_processed.head()

## Testes

In [ ]:
# df_test = df_processed[['resumo', 'area_avaliacao']].copy()

In [ ]:
# def remove_stopwords(text: str) -> str:
#     stop_words = set(stopwords.words('portuguese'))
#     new_text = [item for item in wordpunct_tokenize(text) if item not in stop_words]
#     return " ".join(set(new_text))

In [ ]:
# df_test.resumo = df_test.resumo.apply(remove_stopwords)

In [ ]:
# def lc_stem(text: str) -> str:
#     stemmer = LancasterStemmer()
#     words = [stemmer.stem(word) for word in wordpunct_tokenize(text)]
#     return " ".join(set(words))

In [ ]:
# lem = df_test.copy()
# lem.resumo = lem.resumo.apply(lc_stem)

In [ ]:
# def spc_lem(text: str) -> str:
#     words = [token.lemma_ for token in nlp(text) if token.pos_ != 'PUNCT']
#     return " ".join(set(words))

In [ ]:
# stem = df_test.copy()
# stem.resumo = stem.resumo.apply(spc_lem)

In [ ]:
# def freq_dist(array: list) -> dict:
#     text = ' '.join(array)
#     return FreqDist(wordpunct_tokenize(text))

In [ ]:
# lem.to_pickle(f'./Processed/{year}_lem.pkl')
# stem.to_pickle(f'./Processed/{year}_stem.pkl')